In [1]:
%reload_kedro
trained_model = catalog.load("trained_model")

2023-02-08 10:38:22,438 - kedro.ipython - INFO - Resolved project path as: /home/daniel/git/self-supervised-learning.
To set a different path, run '%reload_kedro <project_root>'
2023-02-08 10:38:22,690 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:339: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/daniel/git/self-supervised-learning/conf/base', '/home/daniel/git/self-supervised-learning/conf/local'] matching the glob pattern(s): ('credentials*', 'credentials*/**', '**/credentials*')
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")

2023-02-08 10:38:22,714 - kedro.ipython - INFO - Kedro project Self-Supervised Learning
2023-02-08 10:38:22,714 - kedro.ipython - INFO - Defined global variable 'context', 'session', 'catalog' and 'pipelines'
2023-02-08 10:38:22,726 - kedro.io.data_catalog - INFO - Loading data from 'trained_model' 

2023-02-08 10:38:22.727 | WARNING  | self_supervised.data_sets.pytorch_model_data_set:_load:42 - CUDA not available, automatically mapping variables to the CPU.


2023-02-08 10:38:23,024 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3.8/site-packages/thop/profile.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) < LooseVersion("1.0.0"):

2023-02-08 10:38:23,025 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3.8/site-packages/thop/profile.py:68: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) >= LooseVersion("1.1.0"):

2023-02-08 10:38:23,286 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":

2023-02-08 10:38:23,287 - py.warnings - WARNING - /home/daniel/git/self-supervised-learning/.venv/lib/python3

2023-02-08 10:38:23.302 | INFO     | self_supervised.pipelines.train_simclr.nodes:<module>:29 - Using cpu device.


In [2]:
import torch

torch.save({"model": trained_model.encoder.yolo}, "simclr_yolov5l.pt")